1- LOAD EL DATASET

2- CHUNCK IT FOR RETRIVAL (TO BE READY FOR SEARCHING FOR SEMANTICALLY RELEVANT DOCS

3- PICK 2 MODELS

4- WAHED (ALREADY PRETRAINED MASLN ON SQUAD (NO NEED FOR FINE TUNING)) (USE IT 3ALA TOL) EX1

5- WAHED (HAN-FINE TUNE IT ON SQUAD, W YKON DIFFERENT ARCHITECTURE (W MESH FAR2A QA WALA GENRAL LM)) EX2

6- PICK AN EVALUATION METRICS W N-COMPARE BOTH (f1 , exact match)

# **EXPERIMENT 1**

Model 1 (Pretrained Only – Baseline)
Example: deepset/roberta-base-squad2

Already trained on SQuAD.

You use it as-is with a QA pipeline.

In [21]:
!pip install langchain datasets sentence-transformers faiss-cpu
!pip install -U transformers
!pip install -U langchain-community
!pip install evaluate


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 54.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
plotnine 0.14.5 requires matplotlib>=3.8.0, but you have matplotlib 3.7.2 which is incompatible.
pandas-gbq 0.28.0 requires google-api-core<3.0.0dev,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
mlxtend 0.23.4 requires scikit-learn>=1.3.1, but you have scikit-learn 1.2.2 which is incompatibl

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.9/437.9 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.50
    Uninstalling langchain-core-0.3.50:
      Successfully uninstalled langchain-core-0.3.50
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.7
    Uninstalling langchain-text-splitters-0.3.7:
      Successfully uninstalled langchain-text-splitters-0.3.7
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.22
    Uninstalling langchain-0.3.22:
      Successfully uninstalled langchain-0.3.22


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [22]:
from datasets import load_dataset
dataset = load_dataset("squad_v2", split="validation")
contexts = dataset["context"]
questions = dataset["question"]
answers = dataset["answers"]
print(f"Loaded {len(contexts)} context passages and {len(questions)} questions.")


README.md:   0%|          | 0.00/8.92k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

Loaded 11873 context passages and 11873 questions.


Wrap Contexts as LangChain Documents
📌 What it does:
Wraps each unique context string as a Document, tagging it with a source ID.


In [23]:
from langchain.schema import Document #LangChain document wrappers

docs = [Document(page_content=ctx, metadata={"source": f"squad_{i}"})
        for i, ctx in enumerate(set(contexts))]  # Remove duplicates and stores only unique context, along with the source id from squad (as a doc object)

Embed and Store with FAISS
📌 What it does:
Creates embeddings for all documents and stores them in a FAISS index for fast similarity search. (FAISS = docs embedded)

In [24]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
db = FAISS.from_documents(docs, embedding_model)
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 3})  # Optional: adjust k this returns the 3 most relevant docs, to pass to the QA model to extract the answer.
print("Semantic retriever is ready.")

/tmp/ipykernel_35/1249088835.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Semantic retriever is ready.


 Load QA Pipeline Model ( STEP 1 IN EXPERIMENT 1)
📌 What it does:
Loads deepset/roberta-base-squad2 into a HuggingFace QA pipeline, and wraps it for LangChain use. And adds it to the pipeline right away.

LLM = typeof pipeline is QA, MODEL IS ROBERTA


In [25]:
from transformers import pipeline
from langchain.llms import HuggingFacePipeline

qa_pipeline = pipeline("question-answering", model="deepset/roberta-base-squad2")
llm = HuggingFacePipeline(pipeline=qa_pipeline)
print("Loaded RoBERTa QA model (extractive).")

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Device set to use cuda:0


Loaded RoBERTa QA model (extractive).


/tmp/ipykernel_35/135334424.py:5: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=qa_pipeline)


Create a LangChain QA Chain with Retrieval
📌 What it does:
Connects the retriever (FAISS + context chunks) with the QA model to form a Retrieval-Augmented Generation (RAG) setup.

STARTS TO CONNECT THE RETRIEVAL WITH THE MODEL, 3an taree2 el pipeline elly ehna lesa m-createno el step elly fatet (llm)


In [26]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)
print("RetrievalQA chain is ready.")

RetrievalQA chain is ready.


## TEST

In [27]:
query = "Tell me about Jay Z and Beyonce"
docs = retriever.get_relevant_documents(query)
# USE ONLY THE TOP MOST RELEVANT DOC
if docs:
    context = "\n".join([doc.page_content for doc in docs])
    result = qa_pipeline(question=query, context=context)
    print("Answer:", result["answer"])
    print("Retrieved context snippet:", context[:200], "...")
else:
    print("No relevant context found.")

Answer: Conan O'Brien
Retrieved context snippet: Shi Tianze was a Han Chinese who lived in the Jin dynasty. Interethnic marriage between Han and Jurchen became common at this time. His father was Shi Bingzhi (史秉直, Shih Ping-chih). Shi Bingzhi was ma ...


/tmp/ipykernel_35/3516660364.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(query)


## EVALUATE

In [31]:
from evaluate import load

# Load the official SQuAD v2 evaluation metric
metric = load("squad")

# Prepare predictions and references
predictions = []
references = []

# Loop over a small subset for demo
for i in range(20):  # Adjust number as needed
    question = questions[i]
    answer_texts = answers[i]['text']
    id_ = dataset[i]['id']

    # 🛠️ Fix: Handle empty ground truths
    if not answer_texts:
        print(f"[DEBUG] Unanswerable question found at index {i}, ID: {id_}")
        answer_texts = [""]

    docs = retriever.get_relevant_documents(question)
    if docs:
        context = docs[0].page_content
        result = qa_pipeline(question=question, context=context)
        prediction_text = result.get("answer", "")
    else:
        prediction_text = ""  # If no context found, treat as no answer

    predictions.append({
        "id": id_,
        "prediction_text": prediction_text
    })

    references.append({
        "id": id_,
        "answers": {
            "text": answer_texts,
            "answer_start": answers[i]["answer_start"]
        }
    })
    
for ref in references:
    if not ref["answers"]["text"]:
        print(f"Empty ground truth for ID: {ref['id']}")



[DEBUG] Unanswerable question found at index 5, ID: 5ad39d53604f3c001a3fe8d1
[DEBUG] Unanswerable question found at index 6, ID: 5ad39d53604f3c001a3fe8d2
[DEBUG] Unanswerable question found at index 7, ID: 5ad39d53604f3c001a3fe8d3
[DEBUG] Unanswerable question found at index 8, ID: 5ad39d53604f3c001a3fe8d4
[DEBUG] Unanswerable question found at index 12, ID: 5ad3a266604f3c001a3fea27
[DEBUG] Unanswerable question found at index 13, ID: 5ad3a266604f3c001a3fea28
[DEBUG] Unanswerable question found at index 14, ID: 5ad3a266604f3c001a3fea29
[DEBUG] Unanswerable question found at index 15, ID: 5ad3a266604f3c001a3fea2a
[DEBUG] Unanswerable question found at index 16, ID: 5ad3a266604f3c001a3fea2b
[DEBUG] Unanswerable question found at index 19, ID: 5ad3ab70604f3c001a3feb89


In [ ]:
# Compute the evaluation
results = metric.compute(predictions=predictions, references=references)
print("Evaluation Results:", results)

# **EXPERIMENT 2**

Model 2 (To Fine-Tune – Experiment): T5 (or another encoder model like albert, distilbert, etc.)

Not yet fine-tuned for QA.

You’ll fine-tune it using the SQuAD training set.

We begin by installing all necessary libraries for fine-tuning the T5 model on a question answering (QA) task. These include:

- `transformers`: Provides pre-trained models like T5 and training utilities.
- `datasets`: Gives access to datasets like SQuAD.
- `torch`: PyTorch framework for model training.
- `accelerate`: Helps with multi-GPU or mixed precision training.
- `evaluate`: For computing Exact Match and F1 Score.


In [3]:
!pip install langchain sentence-transformers faiss-cpu
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 56.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 6.3 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
plotnine 0.14.5 requires matplotlib>=3.8.0, but you have matplotlib 3.7.2 which is incompatible.
pandas-gbq 0.28.0 requires google-api-core<3.0.0dev,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
mlxtend 0.23.4 requires scikit-learn>=1.3.1, but you have scikit-learn 1.2.2 which is incompatibl

In [2]:
!pip install -U transformers
!pip install datasets torch accelerate evaluate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 32.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires num

We check if a GPU is available using `nvidia-smi`. This ensures that the training process will be accelerated.

In [2]:
!nvidia-smi

Sat May 17 18:37:28 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              9W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Load and Prepare SQuAD v2 Dataset: Import all necessary libraries for data loading, model building, retrieval, and evaluation.


In [4]:
from datasets import load_dataset, Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
from langchain.schema import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
import evaluate
import torch
import numpy as np

2025-05-18 12:50:24.102769: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747572624.274292      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747572624.322920      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Select a Subset of SQuAD v2 for Training and Validation: Load the SQuAD v2 dataset and limit the size to 5,000 samples for both training and validation to reduce training time during experimentation.


In [5]:
# Load dataset (use train + val or any split)
raw_dataset = load_dataset("squad_v2")
train_raw = raw_dataset["train"].select(range(5000))
val_raw = raw_dataset["validation"].select(range(5000))

README.md:   0%|          | 0.00/8.92k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/130319 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11873 [00:00<?, ? examples/s]

Create a Dense Retriever with FAISS and Sentence Transformers: Build a FAISS vector store to retrieve relevant context paragraphs for each question using dense embeddings.


In [6]:
# Build retriever from unique contexts
unique_contexts = list(set(train_raw["context"]))
docs = [Document(page_content=ctx, metadata={"source": f"squad_{i}"}) for i, ctx in enumerate(unique_contexts)]

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
db = FAISS.from_documents(docs, embedding_model)
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 3})
print("Retriever is ready.")

/tmp/ipykernel_35/2469915500.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Retriever is ready.


Prepare Input-Target Pairs for the Model Using Retrieved Contexts: Use the retriever to fetch relevant contexts and prepare inputs in the format expected by the T5 model: "question: ... context: ..." → "answer"


In [7]:
# Function to prepare input_texts using retriever
def prepare_data(split):
    questions = split["question"]
    answers = split["answers"]
    input_texts = []
    target_texts = []

    for q, a in zip(questions, answers):
        retrieved_docs = retriever.get_relevant_documents(q)
        if retrieved_docs:
            context = "\n".join([doc.page_content for doc in retrieved_docs])
            input_texts.append(f"question: {q} context: {context}")
            target_texts.append(a["text"][0] if a["text"] else "no answer")
        else:
            input_texts.append(f"question: {q} context: ")
            target_texts.append("no answer")
    
    return Dataset.from_dict({"input_text": input_texts, "target_text": target_texts})

train_data = prepare_data(train_raw)
val_data = prepare_data(val_raw)

/tmp/ipykernel_35/1883711174.py:9: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(q)


Load T5 Tokenizer: Load the tokenizer corresponding to the t5-small model to convert text to token IDs.


In [8]:
# Load tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-small")

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Tokenize Input and Target Texts: Tokenize the inputs and targets for training and evaluation. Ensures consistent sequence length for batching.


In [9]:
# Tokenize
def tokenize(batch):
    inputs = tokenizer(batch["input_text"], padding="max_length", truncation=True, max_length=512)
    targets = tokenizer(batch["target_text"], padding="max_length", truncation=True, max_length=64)
    inputs["labels"] = targets["input_ids"]
    return inputs

train_tokenized = train_data.map(tokenize, batched=True, remove_columns=train_data.column_names)
val_tokenized = val_data.map(tokenize, batched=True, remove_columns=val_data.column_names)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Compute Evaluation Metric (F1 & Exact Match): Compute SQuAD-style F1 and Exact Match metrics by comparing model predictions to ground truth answers.


In [44]:
# Load metric
squad_metric = evaluate.load("squad")
rouge_metric = evaluate.load("rouge")


def compute_metrics(pred):
    # Filter out invalid token IDs (-100 and out-of-range)
    pred_ids = np.where(pred.predictions != -100, pred.predictions, tokenizer.pad_token_id)
    pred_ids = np.clip(pred_ids, 0, tokenizer.vocab_size - 1)  # Ensure IDs are within valid range
    
    label_ids = np.where(pred.label_ids != -100, pred.label_ids, tokenizer.pad_token_id)
    label_ids = np.clip(label_ids, 0, tokenizer.vocab_size - 1)
    
    preds = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels = tokenizer.batch_decode(label_ids, skip_special_tokens=True)
    
    formatted_preds = [{"id": str(i), "prediction_text": p} for i, p in enumerate(preds)]
    formatted_labels = [{"id": str(i), "answers": {"text": [l], "answer_start": [0]}} for i, l in enumerate(labels)]
    
    squad_results = squad_metric.compute(predictions=formatted_preds, references=formatted_labels)
    
    # ROUGE expects lists of references and predictions (strings)
    rouge_results = rouge_metric.compute(predictions=preds, references=labels, use_stemmer=True)
    
    return {
        "f1": squad_results["f1"],
        "exact_match": squad_results["exact_match"],
        "rouge1": rouge_results["rouge1"],
        "rouge2": rouge_results["rouge2"],
        "rougeL": rouge_results["rougeL"],
        "rougeLsum": rouge_results["rougeLsum"],
    }

Set Training Arguments: Define training configurations such as batch size, number of epochs, and whether to use mixed-precision (fp16) if a GPU is available.



In [11]:
# Training setup
training_args = Seq2SeqTrainingArguments(
    output_dir="./t5-qa-retriever",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    predict_with_generate=True,
    logging_steps=100,
    save_total_limit=1,
    report_to=[],
    fp16=torch.cuda.is_available()
)


Load T5 Model and Create Trainer: Initialize the T5ForConditionalGeneration model and wrap it with Seq2SeqTrainer for training and evaluation.


In [12]:
model = T5ForConditionalGeneration.from_pretrained("t5-small")

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/tmp/ipykernel_35/2023422548.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Train the Model: Start fine-tuning the T5 model on the retrieved-question-answer dataset.


In [13]:
trainer.train()

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss
100,3.624500
200,0.170500
300,0.099000
400,0.092100
500,0.077500
600,0.080000
700,0.079400
800,0.074800
900,0.074700


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=939, training_loss=0.4690004976920553, metrics={'train_runtime': 367.1489, 'train_samples_per_second': 40.855, 'train_steps_per_second': 2.558, 'total_flos': 2030127022080000.0, 'train_loss': 0.4690004976920553, 'epoch': 3.0})

Evaluate: We use the same evaluate library to load the official SQuAD metric script as we did in Experiment 1


In [45]:
trainer.compute_metrics = compute_metrics

In [46]:
results = trainer.evaluate()
print(results)

{'eval_loss': 0.24142524600028992, 'eval_f1': 13.991233802614085, 'eval_exact_match': 12.84, 'eval_rouge1': 0.1437745505079004, 'eval_rouge2': 0.1276861904761905, 'eval_rougeL': 0.1429931037267286, 'eval_rougeLsum': 0.14370372154786293, 'eval_runtime': 157.95, 'eval_samples_per_second': 31.656, 'eval_steps_per_second': 1.982, 'epoch': 3.0}
